<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/SensorAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import pickle
import os
from datetime import datetime, timedelta
from collections import deque
from typing import Dict, List, Optional, Tuple, Union
import warnings
warnings.filterwarnings('ignore')

# Core ML libraries
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from scipy import stats
from scipy.spatial.distance import jensenshannon

# Deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.models import Sequential, Model, load_model
    from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Input
    from tensorflow.keras.optimizers import Adam
    KERAS_AVAILABLE = True
except ImportError:
    KERAS_AVAILABLE = False

# Statistical modeling
try:
    from statsmodels.tsa.arima.model import ARIMA
    STATSMODELS_AVAILABLE = True
except ImportError:
    STATSMODELS_AVAILABLE = False


class ProductionSensorAgent:
    """
    Production-ready Sensor Agent for IoT systems.

    Key features:
    - Pre-trained models loaded from disk
    - Processes single subsequences in real-time
    - Handles multi-sensor input [timesteps, num_sensors]
    - Individual sensor scoring and anomaly detection
    - Incremental retraining when needed
    """

    def __init__(self,
                 models_dir: str = "./models",
                 window_length: int = 50,
                 num_sensors: int = 12,
                 memory_size: int = 1000,
                 threshold_k: float = 2.0,
                 drift_threshold: float = 0.1,
                 auto_save: bool = True):
        """
        Initialize Production Sensor Agent.

        Args:
            models_dir: Directory containing pre-trained models
            window_length: Length of time window (timesteps)
            num_sensors: Number of sensors in the system
            memory_size: Size of rolling memory buffer
            threshold_k: Multiplier for adaptive threshold
            drift_threshold: Threshold for drift detection
            auto_save: Automatically save models after retraining
        """
        self.models_dir = models_dir
        self.window_length = window_length
        self.num_sensors = num_sensors
        self.memory_size = memory_size
        self.threshold_k = threshold_k
        self.drift_threshold = drift_threshold
        self.auto_save = auto_save

        # Create models directory if it doesn't exist
        os.makedirs(models_dir, exist_ok=True)

        # Per-sensor components - all stored as lists indexed by sensor_id
        self.models = {}  # sensor_id -> trained model
        self.scalers = {}  # sensor_id -> fitted scaler
        self.error_memory = {}  # sensor_id -> deque of recent errors
        self.data_memory = {}  # sensor_id -> deque of recent data
        self.rolling_stats = {}  # sensor_id -> current statistics
        self.recent_errors = {}  # sensor_id -> recent errors for drift detection

        # Per-sensor state tracking
        self.total_processed = {}
        self.anomalies_detected = {}
        self.drift_detected_count = {}
        self.last_retrain_time = {}
        self.retrain_history = {}

        # Initialize per-sensor containers
        for sensor_id in range(num_sensors):
            self._init_sensor(sensor_id)

        # Load pre-trained models
        self.load_all_models()

    def _init_sensor(self, sensor_id: int):
        """Initialize containers for a single sensor."""
        self.error_memory[sensor_id] = deque(maxlen=self.memory_size)
        self.data_memory[sensor_id] = deque(maxlen=self.memory_size)
        self.recent_errors[sensor_id] = deque(maxlen=100)

        self.rolling_stats[sensor_id] = {
            'mean': 0.0,
            'std': 1.0,
            'q95': 0.0,
            'q99': 0.0
        }

        self.total_processed[sensor_id] = 0
        self.anomalies_detected[sensor_id] = 0
        self.drift_detected_count[sensor_id] = 0
        self.last_retrain_time[sensor_id] = None
        self.retrain_history[sensor_id] = []

    def save_model(self, sensor_id: int, model_type: str = 'lstm_autoencoder'):
        """Save individual sensor model and scaler to disk."""
        if sensor_id not in self.models or sensor_id not in self.scalers:
            print(f"Warning: No trained model/scaler for sensor {sensor_id}")
            return False

        try:
            # Save model
            model_path = os.path.join(self.models_dir, f"sensor_{sensor_id}_model.h5")
            if model_type in ['lstm_autoencoder', 'vae']:
                self.models[sensor_id].save(model_path)

            # Save scaler and metadata
            metadata = {
                'scaler': self.scalers[sensor_id],
                'rolling_stats': self.rolling_stats[sensor_id],
                'model_type': model_type,
                'window_length': self.window_length,
                'created_at': datetime.now(),
                'error_history': list(self.error_memory[sensor_id])[-100:]  # Save recent errors
            }

            metadata_path = os.path.join(self.models_dir, f"sensor_{sensor_id}_metadata.pkl")
            with open(metadata_path, 'wb') as f:
                pickle.dump(metadata, f)

            print(f"Saved model for sensor {sensor_id}")
            return True

        except Exception as e:
            print(f"Failed to save model for sensor {sensor_id}: {e}")
            return False

    def load_model(self, sensor_id: int) -> bool:
        """Load individual sensor model from disk."""
        try:
            model_path = os.path.join(self.models_dir, f"sensor_{sensor_id}_model.h5")
            metadata_path = os.path.join(self.models_dir, f"sensor_{sensor_id}_metadata.pkl")

            if not os.path.exists(model_path) or not os.path.exists(metadata_path):
                print(f"No saved model found for sensor {sensor_id}")
                return False

            # Load model
            self.models[sensor_id] = load_model(model_path)

            # Load metadata
            with open(metadata_path, 'rb') as f:
                metadata = pickle.load(f)

            self.scalers[sensor_id] = metadata['scaler']
            self.rolling_stats[sensor_id] = metadata['rolling_stats']

            # Restore error history for drift detection
            if 'error_history' in metadata:
                self.error_memory[sensor_id].extend(metadata['error_history'])

            print(f"Loaded model for sensor {sensor_id}")
            return True

        except Exception as e:
            print(f"Failed to load model for sensor {sensor_id}: {e}")
            return False

    def load_all_models(self):
        """Load all pre-trained sensor models."""
        print("Loading pre-trained sensor models...")
        loaded_count = 0

        for sensor_id in range(self.num_sensors):
            if self.load_model(sensor_id):
                loaded_count += 1

        print(f"Loaded {loaded_count}/{self.num_sensors} sensor models")

        if loaded_count == 0:
            print("WARNING: No pre-trained models found. You need to train models first!")

    def _compute_anomaly_score(self, sensor_data: np.ndarray, sensor_id: int) -> float:
        """Compute anomaly score for a single sensor's data."""
        if sensor_id not in self.models or sensor_id not in self.scalers:
            print(f"Warning: No model loaded for sensor {sensor_id}")
            return 0.0

        try:
            # Normalize the sensor data
            data_scaled = self.scalers[sensor_id].transform(sensor_data.reshape(-1, 1)).flatten()

            # Reshape for model input: [1, timesteps, 1]
            X = data_scaled.reshape(1, len(data_scaled), 1)

            # Get reconstruction
            reconstruction = self.models[sensor_id].predict(X, verbose=0)
            reconstruction = reconstruction.reshape(-1)

            # Compute reconstruction error (MSE)
            error = mean_squared_error(data_scaled, reconstruction)

            return float(error)

        except Exception as e:
            print(f"Error computing anomaly score for sensor {sensor_id}: {e}")
            return 0.0

    def _update_rolling_stats(self, sensor_id: int, errors: List[float]):
        """Update rolling statistics for a sensor."""
        if len(errors) == 0:
            return

        errors_array = np.array(errors)
        self.rolling_stats[sensor_id]['mean'] = np.mean(errors_array)
        self.rolling_stats[sensor_id]['std'] = np.std(errors_array) + 1e-8
        self.rolling_stats[sensor_id]['q95'] = np.percentile(errors_array, 95)
        self.rolling_stats[sensor_id]['q99'] = np.percentile(errors_array, 99)

    def _detect_drift(self, sensor_id: int) -> Tuple[bool, float]:
        """Detect distribution drift for a specific sensor."""
        if len(self.error_memory[sensor_id]) < 100 or len(self.recent_errors[sensor_id]) < 30:
            return False, 0.0

        try:
            # Compare recent errors vs historical errors
            historical_errors = np.array(list(self.error_memory[sensor_id])[:-50])  # Exclude recent
            recent_errors = np.array(list(self.recent_errors[sensor_id]))

            # Jensen-Shannon divergence
            hist_baseline, bins = np.histogram(historical_errors, bins=20, density=True)
            hist_recent, _ = np.histogram(recent_errors, bins=bins, density=True)

            hist_baseline += 1e-10
            hist_recent += 1e-10
            hist_baseline /= hist_baseline.sum()
            hist_recent /= hist_recent.sum()

            js_divergence = jensenshannon(hist_baseline, hist_recent)
            drift_detected = js_divergence > self.drift_threshold

            return drift_detected, float(js_divergence)

        except Exception:
            # Fallback to KS test
            try:
                historical_errors = list(self.error_memory[sensor_id])[:-50]
                recent_errors = list(self.recent_errors[sensor_id])
                ks_stat, p_value = stats.ks_2samp(historical_errors, recent_errors)
                return p_value < 0.05, float(ks_stat)
            except:
                return False, 0.0

    def process_subsequence(self, subsequence: np.ndarray, timestamp: datetime = None) -> Dict:
        """
        Process a single subsequence from all sensors.

        Args:
            subsequence: Input data shape [window_length, num_sensors]
            timestamp: Timestamp for this subsequence

        Returns:
            Dictionary with results for all sensors
        """
        if timestamp is None:
            timestamp = datetime.now()

        if subsequence.shape != (self.window_length, self.num_sensors):
            raise ValueError(f"Expected shape ({self.window_length}, {self.num_sensors}), "
                           f"got {subsequence.shape}")

        results = {
            "timestamp": timestamp,
            "global_summary": {},
            "sensor_results": {}
        }

        global_anomalies = 0
        global_drift_count = 0

        # Process each sensor individually
        for sensor_id in range(self.num_sensors):
            # Extract data for this sensor
            sensor_data = subsequence[:, sensor_id]  # Shape: [window_length]

            # Compute anomaly score
            anomaly_score = self._compute_anomaly_score(sensor_data, sensor_id)

            # Update memory
            self.data_memory[sensor_id].append(sensor_data.copy())
            self.error_memory[sensor_id].append(anomaly_score)
            self.recent_errors[sensor_id].append(anomaly_score)

            # Adaptive thresholding
            stats = self.rolling_stats[sensor_id]
            current_threshold = stats['mean'] + self.threshold_k * stats['std']
            is_anomaly = anomaly_score > current_threshold

            # Confidence calculation
            confidence = min(1.0, abs(anomaly_score - current_threshold) / (stats['std'] + 1e-8))

            # Drift detection
            drift_flag, drift_score = self._detect_drift(sensor_id)

            # Update counters
            self.total_processed[sensor_id] += 1
            if is_anomaly:
                self.anomalies_detected[sensor_id] += 1
                global_anomalies += 1

            if drift_flag:
                self.drift_detected_count[sensor_id] += 1
                global_drift_count += 1

            # Update statistics periodically
            if len(self.error_memory[sensor_id]) >= 50 and len(self.error_memory[sensor_id]) % 10 == 0:
                self._update_rolling_stats(sensor_id, list(self.error_memory[sensor_id])[-50:])

            # Store sensor results
            results["sensor_results"][sensor_id] = {
                "sensor_id": sensor_id,
                "anomaly_score": float(anomaly_score),
                "is_anomaly": bool(is_anomaly),
                "drift_flag": bool(drift_flag),
                "confidence": float(confidence),
                "threshold_used": float(current_threshold),
                "drift_score": float(drift_score),
                "anomaly_rate": self.anomalies_detected[sensor_id] / max(1, self.total_processed[sensor_id]),
                "needs_retrain": self._should_retrain(sensor_id)
            }

        # Global summary
        results["global_summary"] = {
            "total_sensors": self.num_sensors,
            "anomalous_sensors": global_anomalies,
            "drift_sensors": global_drift_count,
            "global_anomaly_rate": global_anomalies / self.num_sensors,
            "sensors_needing_retrain": sum(1 for sid in range(self.num_sensors) if self._should_retrain(sid))
        }

        return results

    def _should_retrain(self, sensor_id: int) -> bool:
        """Check if a specific sensor needs retraining."""
        if self.last_retrain_time[sensor_id] is None:
            return False

        time_since_retrain = datetime.now() - self.last_retrain_time[sensor_id]
        drift_rate = self.drift_detected_count[sensor_id] / max(1, self.total_processed[sensor_id])

        return (time_since_retrain.days > 7 or  # Weekly retraining
                drift_rate > 0.1 or  # High drift rate
                (len(self.error_memory[sensor_id]) > 500 and
                 np.mean(list(self.error_memory[sensor_id])[-100:]) > 2 * self.rolling_stats[sensor_id]['mean']))

    def retrain_sensor(self, sensor_id: int, incremental_data: np.ndarray, reason: str = "incremental") -> bool:
        """
        Retrain a specific sensor with incremental data.

        Args:
            sensor_id: Which sensor to retrain
            incremental_data: New training data [n_samples, window_length] or [total_timesteps]
            reason: Reason for retraining

        Returns:
            True if retraining successful
        """
        print(f"RETRAINING SENSOR {sensor_id} - Reason: {reason}")

        try:
            # Prepare training data
            if incremental_data.ndim == 1:
                # Convert flat time series to sequences
                sequences = []
                for i in range(len(incremental_data) - self.window_length + 1):
                    sequences.append(incremental_data[i:i + self.window_length])
                training_sequences = np.array(sequences)
            else:
                # Already in sequence format
                training_sequences = incremental_data

            # Add memory data for stability
            if len(self.data_memory[sensor_id]) > 0:
                memory_sequences = np.array(list(self.data_memory[sensor_id]))
                training_sequences = np.vstack([memory_sequences, training_sequences])

            if len(training_sequences) < 50:
                print(f"Insufficient data for retraining sensor {sensor_id}")
                return False

            # Retrain scaler
            flat_data = training_sequences.flatten()
            self.scalers[sensor_id] = StandardScaler()
            scaled_data = self.scalers[sensor_id].fit_transform(flat_data.reshape(-1, 1)).flatten()

            # Prepare sequences for model training
            X_train = []
            for seq in training_sequences:
                seq_scaled = self.scalers[sensor_id].transform(seq.reshape(-1, 1)).flatten()
                X_train.append(seq_scaled)

            X_train = np.array(X_train).reshape(len(X_train), self.window_length, 1)

            # Retrain model (create new or retrain existing)
            if sensor_id not in self.models:
                self.models[sensor_id] = self._build_lstm_autoencoder()

            # Train model
            self.models[sensor_id].fit(X_train, X_train, epochs=30, batch_size=16, verbose=0)

            # Update statistics
            errors = []
            for seq in X_train:
                pred = self.models[sensor_id].predict(seq.reshape(1, self.window_length, 1), verbose=0)
                error = mean_squared_error(seq.flatten(), pred.flatten())
                errors.append(error)

            self._update_rolling_stats(sensor_id, errors)

            # Reset counters
            self.drift_detected_count[sensor_id] = 0
            self.recent_errors[sensor_id].clear()
            self.last_retrain_time[sensor_id] = datetime.now()

            # Save model if auto_save enabled
            if self.auto_save:
                self.save_model(sensor_id)

            print(f"✅ SENSOR {sensor_id} RETRAINED SUCCESSFULLY")
            return True

        except Exception as e:
            print(f"❌ SENSOR {sensor_id} RETRAINING FAILED: {e}")
            return False

    def _build_lstm_autoencoder(self) -> Model:
        """Build LSTM Autoencoder for a sensor."""
        if not KERAS_AVAILABLE:
            raise ImportError("Keras/TensorFlow required")

        inputs = Input(shape=(self.window_length, 1))
        encoded = LSTM(32, activation='relu', return_sequences=False)(inputs)
        decoded = RepeatVector(self.window_length)(encoded)
        decoded = LSTM(32, activation='relu', return_sequences=True)(decoded)
        outputs = TimeDistributed(Dense(1, activation='linear'))(decoded)

        model = Model(inputs, outputs)
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
        return model

    def get_sensor_status(self, sensor_id: int) -> Dict:
        """Get detailed status for a specific sensor."""
        if sensor_id not in range(self.num_sensors):
            return {"error": "Invalid sensor_id"}

        return {
            "sensor_id": sensor_id,
            "has_model": sensor_id in self.models,
            "total_processed": self.total_processed[sensor_id],
            "anomalies_detected": self.anomalies_detected[sensor_id],
            "anomaly_rate": self.anomalies_detected[sensor_id] / max(1, self.total_processed[sensor_id]),
            "drift_events": self.drift_detected_count[sensor_id],
            "memory_usage": len(self.error_memory[sensor_id]) / self.memory_size,
            "needs_retrain": self._should_retrain(sensor_id),
            "last_retrain": self.last_retrain_time[sensor_id],
            "rolling_stats": self.rolling_stats[sensor_id].copy()
        }

    def get_system_status(self) -> Dict:
        """Get overall system status."""
        loaded_models = sum(1 for sid in range(self.num_sensors) if sid in self.models)
        needing_retrain = sum(1 for sid in range(self.num_sensors) if self._should_retrain(sid))
        total_processed = sum(self.total_processed.values())
        total_anomalies = sum(self.anomalies_detected.values())

        return {
            "total_sensors": self.num_sensors,
            "loaded_models": loaded_models,
            "sensors_needing_retrain": needing_retrain,
            "total_processed": total_processed,
            "total_anomalies": total_anomalies,
            "global_anomaly_rate": total_anomalies / max(1, total_processed),
            "models_dir": self.models_dir,
            "system_ready": loaded_models == self.num_sensors
        }


# Pre-training utility for initial model creation
def pretrain_sensor_models(training_data: Dict[int, np.ndarray],
                          models_dir: str = "./models",
                          window_length: int = 50) -> ProductionSensorAgent:
    """
    Pre-train models for all sensors and save them.

    Args:
        training_data: Dict mapping sensor_id -> historical time series
        models_dir: Directory to save models
        window_length: Window length for sequences

    Returns:
        Configured ProductionSensorAgent with trained models
    """
    print("=== PRE-TRAINING SENSOR MODELS ===")

    num_sensors = len(training_data)
    agent = ProductionSensorAgent(models_dir=models_dir,
                                window_length=window_length,
                                num_sensors=num_sensors,
                                auto_save=False)  # We'll save manually

    for sensor_id, data in training_data.items():
        print(f"Training sensor {sensor_id}...")

        # Create sequences from training data
        sequences = []
        for i in range(len(data) - window_length + 1):
            sequences.append(data[i:i + window_length])

        if len(sequences) < 100:
            print(f"Warning: Limited training data for sensor {sensor_id}")
            continue

        training_sequences = np.array(sequences)

        # Fit scaler
        flat_data = training_sequences.flatten()
        agent.scalers[sensor_id] = StandardScaler()
        scaled_data = agent.scalers[sensor_id].fit_transform(flat_data.reshape(-1, 1)).flatten()

        # Prepare training data
        X_train = []
        for seq in training_sequences:
            seq_scaled = agent.scalers[sensor_id].transform(seq.reshape(-1, 1)).flatten()
            X_train.append(seq_scaled)

        X_train = np.array(X_train).reshape(len(X_train), window_length, 1)

        # Build and train model
        agent.models[sensor_id] = agent._build_lstm_autoencoder()
        agent.models[sensor_id].fit(X_train, X_train, epochs=50, batch_size=32, verbose=0)

        # Initialize statistics
        errors = []
        for seq in X_train[-50:]:  # Use last 50 for stats
            pred = agent.models[sensor_id].predict(seq.reshape(1, window_length, 1), verbose=0)
            error = mean_squared_error(seq.flatten(), pred.flatten())
            errors.append(error)

        agent._update_rolling_stats(sensor_id, errors)
        agent.error_memory[sensor_id].extend(errors)
        agent.last_retrain_time[sensor_id] = datetime.now()

        # Save model
        agent.save_model(sensor_id)
        print(f"✅ Sensor {sensor_id} trained and saved")

    print(f"=== PRE-TRAINING COMPLETE: {len(agent.models)}/{num_sensors} models ===")
    return agent


# Example usage for production deployment
def main_production_example():
    """Example of production usage."""
    print("=== PRODUCTION SENSOR AGENT DEMO ===\n")

    # 1. PRE-TRAINING PHASE (Run once, before deployment)
    print("1. Pre-training phase...")

    # Simulate historical training data for each sensor
    training_data = {}
    for sensor_id in range(12):  # 12 sensors
        # Generate diverse training data per sensor
        t = np.linspace(0, 10*np.pi, 2000)
        signal = (np.sin(t + sensor_id) + 0.5*np.sin(3*t + sensor_id) +
                 np.random.normal(0, 0.1, len(t)))
        training_data[sensor_id] = signal

    # Pre-train and save models
    agent = pretrain_sensor_models(training_data, models_dir="./production_models")

    # 2. PRODUCTION DEPLOYMENT (Load pre-trained models)
    print(f"\n2. Production deployment...")
    production_agent = ProductionSensorAgent(
        models_dir="./production_models",
        window_length=50,
        num_sensors=12
    )

    print(f"System status: {production_agent.get_system_status()}")

    # 3. LIVE PROCESSING (Receive subsequences in real-time)
    print(f"\n3. Processing live data...")

    # Simulate live data stream - shape [window_length, num_sensors]
    for step in range(100):
        # Generate live subsequence
        live_subsequence = np.random.randn(50, 12) + np.sin(np.linspace(0, 2*np.pi, 50)).reshape(-1, 1)

        # Add some anomalies
        if step in [25, 50, 75]:
            live_subsequence[:, step % 12] += 5  # Anomaly in one sensor

        # Process the subsequence
        results = production_agent.process_subsequence(live_subsequence)

        # Check for anomalies
        if results["global_summary"]["anomalous_sensors"] > 0:
            print(f"Step {step}: {results['global_summary']['anomalous_sensors']} anomalous sensors detected")

            # Check individual sensor results
            for sensor_id, sensor_result in results["sensor_results"].items():
                if sensor_result["is_anomaly"]:
                    print(f"  Sensor {sensor_id}: Anomaly score {sensor_result['anomaly_score']:.3f}")

        # Check for retraining needs
        if results["global_summary"]["sensors_needing_retrain"] > 0:
            for sensor_id, sensor_result in results["sensor_results"].items():
                if sensor_result["needs_retrain"]:
                    print(f"🔄 Sensor {sensor_id} needs retraining")

                    # Generate incremental training data
                    new_data = np.random.randn(200) + np.sin(np.linspace(0, 4*np.pi, 200))
                    success = production_agent.retrain_sensor(sensor_id, new_data, "drift_detected")

                    if success:
                        print(f"✅ Sensor {sensor_id} retrained successfully")

    # 4. FINAL STATUS
    print(f"\n4. Final system status:")
    system_status = production_agent.get_system_status()
    for key, value in system_status.items():
        print(f"  {key}: {value}")


if __name__ == "__main__":
    main_production_example()

=== IoT Sensor Agent Framework Demo ===

1. Generating simulated sensor data...
2. Initializing sensor agents...
3. Training agents on historical data...
Sensor sensor_01: Training lstm_autoencoder model...
Sensor sensor_01: Model training completed
Sensor sensor_02: Training lstm_autoencoder model...
Sensor sensor_02: Model training completed
Sensor sensor_03: Training lstm_autoencoder model...
Sensor sensor_03: Model training completed
Sensor sensor_04: Training lstm_autoencoder model...
Sensor sensor_04: Model training completed
Sensor sensor_05: Training lstm_autoencoder model...
Sensor sensor_05: Model training completed
Sensor sensor_06: Training lstm_autoencoder model...
Sensor sensor_06: Model training completed
Sensor sensor_07: Training lstm_autoencoder model...
Sensor sensor_07: Model training completed
Sensor sensor_08: Training lstm_autoencoder model...
Sensor sensor_08: Model training completed
Sensor sensor_09: Training lstm_autoencoder model...
Sensor sensor_09: Model t

Streaming output truncated to the last 5000 lines.
  sensor_11: ANOMALY (score: 0.150, conf: 1.00)
  sensor_12: ANOMALY (score: 0.109, conf: 1.00)
Time 1070: 12 agents detected issues
  sensor_01: ANOMALY (score: 0.077, conf: 1.00)
  sensor_02: ANOMALY (score: 0.205, conf: 1.00)
  sensor_03: ANOMALY (score: 0.326, conf: 1.00)
  sensor_04: ANOMALY (score: 0.031, conf: 1.00)
  sensor_05: ANOMALY (score: 0.427, conf: 1.00)
  sensor_06: ANOMALY (score: 0.026, conf: 1.00)
  sensor_07: ANOMALY (score: 0.074, conf: 1.00)
  sensor_08: ANOMALY (score: 0.199, conf: 1.00)
  sensor_09: ANOMALY (score: 0.074, conf: 1.00)
  sensor_10: ANOMALY (score: 0.391, conf: 1.00)
  sensor_11: ANOMALY (score: 0.150, conf: 1.00)
  sensor_12: ANOMALY (score: 0.112, conf: 1.00)
Time 1071: 12 agents detected issues
  sensor_01: ANOMALY (score: 0.079, conf: 1.00)
  sensor_02: ANOMALY (score: 0.208, conf: 1.00)
  sensor_03: ANOMALY (score: 0.325, conf: 1.00)
  sensor_04: ANOMALY (score: 0.030, conf: 1.00)
  sensor_05

In [ ]:
from google.colab import drive
drive.mount('/content/drive')